In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## Master Template for processing each dataset
- Sentinel-2 (in order of priority)
 - **L2A via GCloud**: in testing
 - **L1C via GCloud processed to L2A**: under development
 - L1C via ESA processed to L2A: partially tested
 - L2A via ESA: partially tested
 - L1C via GCloud: partially tested
 - L1C via ESA: partiall tested

In [6]:
import glob
import shutil
from datetime import datetime
import pandas as pd
import time
import tqdm
import os

from subprocess import Popen, PIPE, STDOUT

from utils.prepS2 import prepareS2


In [7]:
des_scenes_list = ['S2A_MSIL2A_20190812T235741_N0213_R030_T56LRR_20190813T014708', 'S2B_MSIL2A_20190807T235749_N0213_R030_T56MQS_20190808T014706', 'S2B_MSIL2A_20190807T235749_N0213_R030_T56MQT_20190808T014706', 'S2A_MSIL2A_20190812T235741_N0213_R030_T56MRS_20190813T014708', 'S2B_MSIL2A_20190807T235749_N0213_R030_T56MRT_20190808T014706', 'S2A_MSIL2A_20190812T235741_N0213_R030_T57LTL_20190813T032843', 'S2A_MSIL2A_20190812T235741_N0213_R030_T57LUL_20190813T032843', 'S2B_MSIL2A_20190225T234739_N0211_R130_T57LVK_20190226T032107', 'S2B_MSIL2A_20190225T234739_N0211_R130_T57LVL_20190226T032107', 'S2B_MSIL2A_20190225T234739_N0211_R130_T57LWH_20190226T032107', 'S2B_MSIL2A_20190225T234739_N0211_R130_T57LWK_20190226T032107', 'S2A_MSIL2A_20190809T234751_N0213_R130_T57LWL_20190810T015000', 'S2A_MSIL2A_20190227T233811_N0211_R087_T57LXG_20190228T013900', 'S2B_MSIL2A_20190222T233819_N0211_R087_T57LXH_20190223T012250', 'S2B_MSIL2A_20190225T234739_N0211_R130_T57LXK_20190226T032107', 'S2B_MSIL2A_20190225T234739_N0211_R130_T57LXL_20190226T032107', 'S2B_MSIL2A_20190612T233819_N0212_R087_T57LYJ_20190613T013313', 'S2B_MSIL2A_20190622T233829_N0212_R087_T57LYK_20190623T013342', 'S2B_MSIL2A_20190513T233829_N0212_R087_T57LYL_20190514T012938', 'S2A_MSIL2A_20190224T232841_N0211_R044_T57LZH_20190225T012128', 'S2B_MSIL2A_20190110T232839_N0211_R044_T57LZJ_20190111T010849', 'S2B_MSIL2A_20190311T232839_N0211_R044_T57LZK_20190312T011107', 'S2A_MSIL2A_20190812T235741_N0213_R030_T57MTM_20190813T032843', 'S2A_MSIL2A_20190812T235741_N0213_R030_T57MTN_20190813T014708', 'S2A_MSIL2A_20190302T234741_N0211_R130_T57MUM_20190303T014105', 'S2A_MSIL2A_20190812T235741_N0213_R030_T57MUN_20190813T014708', 'S2B_MSIL2A_20190817T235749_N0213_R030_T57MVM_20190818T015053', 'S2B_MSIL2A_20190516T234749_N0212_R130_T57MWM_20190517T014032', 'S2B_MSIL2A_20190406T234749_N0211_R130_T57MWP_20190407T014656', 'S2B_MSIL2A_20190406T234749_N0211_R130_T57MWQ_20190407T014656', 'S2A_MSIL2A_20190224T232841_N0211_R044_T58LBN_20190225T012128', 'S2B_MSIL2A_20190110T232839_N0211_R044_T58LBP_20190111T010849', 'S2B_MSIL2A_20190702T233829_N0212_R087_T58LBR_20190703T013135', 'S2A_MSIL2A_20190303T231851_N0211_R001_T58LEN_20190304T011207', 'S2B_MSIL2A_20190808T232849_N0213_R044_T58LEP_20190809T012044', 'S2A_MSIL2A_20190621T231901_N0212_R001_T58LFN_20190622T010620', 'S2A_MSIL2A_20190621T231901_N0212_R001_T58LFP_20190622T010620', 'S2A_MSIL2A_20190621T231901_N0212_R001_T58LGN_20190622T010620', 'S2A_MSIL2A_20190621T231901_N0212_R001_T58LGP_20190622T010620', 'S2A_MSIL2A_20190529T230911_N0212_R101_T58LGQ_20190530T010056']


In [8]:
os.environ['COPERNICUS_USERNAME']="..."
os.environ['COPERNICUS_PWD']="..."

os.environ['AWS_ACCESS']=""
os.environ['AWS_SECRET']=""

In [ ]:
# output directory currently not cloud bucket, it's just used as ref for intermediate products.
# these test scenes are hardcoded to be uploaded to: public-eo-data/fiji/Sentinel_2_test/
for scene_nm in tqdm.tqdm(des_scenes_list[:2]):
     prepareS2(scene_nm, s3_bucket='public-eo-data', s3_dir='fiji/Sentinel_2_test/', inter_dir='../S2_ARDn/')


  0%|          | 0/2 [00:00<?, ?it/s]

../S2_ARDn/S2A_MSIL2A_20190812T235741_T56LRR_tmp/S2A_MSIL2A_20190812T235741_N0213_R030_T56LRR_20190813T014708.SAFE/GRANULE/L2A_T56LRR_A021620_20190812T235743/IMG_DATA/R10m/T56LRR_20190812T235741_AOT_10m.jp2 ../S2_ARDn/S2A_MSIL2A_20190812T235741_T56LRR_tmp/S2A_MSIL2A_20190812T235741_T56LRR/S2A_MSIL2A_20190812T235741_T56LRR_AOT_10m.tif
../S2_ARDn/S2A_MSIL2A_20190812T235741_T56LRR_tmp/S2A_MSIL2A_20190812T235741_N0213_R030_T56LRR_20190813T014708.SAFE/GRANULE/L2A_T56LRR_A021620_20190812T235743/IMG_DATA/R10m/T56LRR_20190812T235741_B02_10m.jp2 ../S2_ARDn/S2A_MSIL2A_20190812T235741_T56LRR_tmp/S2A_MSIL2A_20190812T235741_T56LRR/S2A_MSIL2A_20190812T235741_T56LRR_B02_10m.tif
../S2_ARDn/S2A_MSIL2A_20190812T235741_T56LRR_tmp/S2A_MSIL2A_20190812T235741_N0213_R030_T56LRR_20190813T014708.SAFE/GRANULE/L2A_T56LRR_A021620_20190812T235743/IMG_DATA/R10m/T56LRR_20190812T235741_B03_10m.jp2 ../S2_ARDn/S2A_MSIL2A_20190812T235741_T56LRR_tmp/S2A_MSIL2A_20190812T235741_T56LRR/S2A_MSIL2A_20190812T235741_T56LRR_B03_


 50%|█████     | 1/2 [02:46<02:46, 166.55s/it]

b"removed '../S2_ARDn/S2A_MSIL2A_20190812T235741_T56LRR_tmp/S2A_MSIL2A_20190812T235741_N0213_R030_T56LRR_20190813T014708.SAFE/DATASTRIP/DS_SGS__20190813T014708_S20190812T235743/MTD_DS.xml'\nremoved '../S2_ARDn/S2A_MSIL2A_20190812T235741_T56LRR_tmp/S2A_MSIL2A_20190812T235741_N0213_R030_T56LRR_20190813T014708.SAFE/DATASTRIP/DS_SGS__20190813T014708_S20190812T235743/QI_DATA/FORMAT_CORRECTNESS.xml'\nremoved '../S2_ARDn/S2A_MSIL2A_20190812T235741_T56LRR_tmp/S2A_MSIL2A_20190812T235741_N0213_R030_T56LRR_20190813T014708.SAFE/DATASTRIP/DS_SGS__20190813T014708_S20190812T235743/QI_DATA/GENERAL_QUALITY.xml'\nremoved '../S2_ARDn/S2A_MSIL2A_20190812T235741_T56LRR_tmp/S2A_MSIL2A_20190812T235741_N0213_R030_T56LRR_20190813T014708.SAFE/DATASTRIP/DS_SGS__20190813T014708_S20190812T235743/QI_DATA/GEOMETRIC_QUALITY.xml'\nremoved '../S2_ARDn/S2A_MSIL2A_20190812T235741_T56LRR_tmp/S2A_MSIL2A_20190812T235741_N0213_R030_T56LRR_20190813T014708.SAFE/DATASTRIP/DS_SGS__20190813T014708_S20190812T235743/QI_DATA/RADIOME

Process ForkPoolWorker-39:
Process ForkPoolWorker-41:
Process ForkPoolWorker-40:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/tj/miniconda3/envs/cs/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process ForkPoolWorker-38:
  File "/home/tj/miniconda3/envs/cs/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/tj/miniconda3/envs/cs/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process ForkPoolWorker-37:
  File "/home/tj/miniconda3/envs/cs/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/tj/miniconda3/envs/cs/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/tj/miniconda3/envs/cs/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self